# Create a notebook to perform Runge-Kutta integration for multiple coupled variables

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define our coupled derivatives to integrate

In [8]:
def dydx(x,y):
    
    # Set the derivatives 
    
    # Our equation is d^2y/dx^2 = -y
    
    # So we can write
    # dydx = z
    # dzdx = -y
    
    # We will set y = y[0]
    # We will sey z = y[1]
    
    # Declare an array
    y_derivs = np.zeros(2)
    
    # Set dydx = x
    y_derivs[0] = y[1]
    
    # Set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    # Here we have to return the arrays of dydx
    return y_derivs

## Define the 4th order RK method

In [9]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    # Declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    # Define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    # Define x at 1 step 
    x_ipo = xi + h
    
    # Declare a tempy array
    y_temp = np.zeros(nv)
    
    # Get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    # Get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    # Get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    # Get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    # Advance y by step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    # THIS IS AN ARRAY
    return yipo

## Define an adaptive step size driver for RK4

In [10]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    # Define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    # Set a maximum number of iterations
    imax = 10000
    
    # Set an iteration variable
    i = 0 
    
    # Create an error 
    Delta = np.full(nv,2*tol)
    
    # Remember the step
    h_step = h
    
    # Adjust the step
    while(Delta.max()/tol > 1.0):
        
        # Estimate our error by taking one step of size h
        # vs. two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_i,nv,0.5*h_step)
        
        # Compute an error
        Delta = np.fabs(y_2 - y_11)
        
        # If error is too latge, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            # Our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        # Check the iteration
        if(i>imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        # Iterate
        i += 1 
        
    # Next time, try to take a bigger step
    h_new= np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_PAC)
    
    # Return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    # dydx is the derivative wrt x
    # a is the lower bound
    # b is the upper bound 
    # y_a are the boundary conditions
    # tol is the tolerance for integrating y
    
    # Define our starting step
    xi = a
    yi = y_a.cop()
    
    # An initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    # Set a maximum number of iterations
    imax = 10000
    
    # Set an iteration variable
    i = 0 
    
    # Set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    # Set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    # Set a flag
    flag = 1
    
    # Loop until we reach the right side
    while(flag):
        
        # Calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,di,yi,nv,h,tol)
        
        #Break
        flag = 0 
        
    